## Installed Required Packages

In [64]:
import sys
!{sys.executable} -m pip install fake-useragent

In [235]:
!{sys.executable} -m pip install BeautifulSoup

    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/private/var/folders/pd/lj_g7cpj6nq6jqn0_47v2fxc0000gn/T/pip-build-6a69sbyc/BeautifulSoup/setup.py", line 22
        print "Unit tests have failed!"
                                      ^
    SyntaxError: Missing parentheses in call to 'print'. Did you mean print(int "Unit tests have failed!")?
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /private/var/folders/pd/lj_g7cpj6nq6jqn0_47v2fxc0000gn/T/pip-build-6a69sbyc/BeautifulSoup/


In [ ]:
!{sys.executable} -m pip install selenium

Download Chrome WebDriver: https://sites.google.com/a/chromium.org/chromedriver/getting-started

Then include the ChromeDriver location in your PATH environment variable

In [65]:
!{sys.executable} -m pip install pymongo

    100% |████████████████████████████████| 317kB 3.0MB/s ta 0:00:01


## Web Crapper Functions

In [2]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import re
def craw_one_page(url):
    user_agent = UserAgent()
    page = requests.get(url,headers={'user-agent':user_agent.chrome})
    content=page.content.decode('utf-8')
    soup = BeautifulSoup(content,'lxml')
    
    dict_result={}
    
    source=soup.find_all('div',attrs={'pane-header'})
    tabs = [div.string for div in source]
    tabs=tabs[:-2]
    
    
    source=soup.find_all('div',attrs={'text-content'})
    tabs_values = [div.string for div in source]
    
    for i in range(len(tabs)):
        dict_result[tabs[i]]=tabs_values[i]
    
    source=soup.find_all('a',attrs={'i'})
    keys = [div["title"] for div in source]
    
    source=soup.find_all('div',attrs={'s_val'})
    values = [div["title"] for div in source]
    
    for i in range(len(keys)):
        dict_result[keys[i]]=values[i]
    return dict_result

In [ ]:
def get_number_of_search_result(source):
    soup=BeautifulSoup(source,'lxml')
    source=soup.find('p', style={"font-size:14px"})
    #for div in source:
    number_link=source.get_text()[36:-15]
    number_link=number_link.replace(",", "")
    return int (number_link)

In [47]:
import time 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
def crawl_search_source(search_url):
    browser=webdriver.Chrome()
    browser.get(search_url)
    number_search_result=get_number_of_search_result(browser.page_source)
    number_500=number_search_result//500
    number_100=(number_search_result-number_500*500)//100
    number_20=(number_search_result-number_500*500-number_100*100)//20
    print (number_500, number_100, number_20)
    for i in range(number_500):
        try:
            time.sleep(1)
            ele=browser.find_element_by_id("button_next500")
            try:
                ele.click()
                #print ("*")
            except:
                time.sleep(0.1)
                i=i-1
        except:
            pass
    for i in range(number_100):
        try:
            time.sleep(1)
            ele=browser.find_element_by_id("button_next100")
            try:
                ele.click()
            except:
                time.sleep(0.1)
                i=i-1
        except:
            pass
    for i in range(number_20):
        try:
            time.sleep(1)
            ele=browser.find_element_by_id("button_next20")
            try:
                ele.click()
            except:
                time.sleep(0.1)
                i=i-1
        except:
            pass
    return browser.page_source, number_search_result

In [36]:
from collections import OrderedDict
def find_links_from_source(soup, number_search_result):
    Tagged_text=soup.find_all('a',target={'_blank'}, )
    set_url=[]
    for x in Tagged_text:
        url=x["href"]
        if (url[0:14]=="/plusd/cables/"):
            set_url.append(x["href"])
    set_url=delete_duplicate(set_url)
    return set_url[0:number_search_result]

In [6]:
import json
def save_obj(data):
    with open('data_crawled.json', 'w') as fp:
        json.dump(data, fp)

In [39]:
def crawl_search_result(set_url):
    url_head= 'https://search.wikileaks.org'
    final_result={}
    i=0
    print ("total number of links: {}".format(len(set_url)))
    for x in set_url:
        print (x[14:-7], "{} out of {} jobs done".format(i+1, len(set_url)))
#         collection.insert_one(craw_one_page(url_head+x))
        final_result[x[14:-7]]=craw_one_page(url_head+x)
        if (i//1000==0):
            save_obj(final_result)
        i=i+1
    save_obj(final_result)

In [1]:
import pickle
def save_pickle(filename, obj):
    with open('{}.pickle'.format(filename), 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
def open_pickle(filename):
    with open('{}.pickle'.format(filename), 'rb') as handle:
        return pickle.load(handle)

In [26]:
def delete_duplicate(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [40]:
search_url="https://search.wikileaks.org/plusd/?qproject[]=ps&qproject[]=cg&qproject[]=cc&qproject[]=fp&qproject[]=ee&q=&qtfrom=2002-01-01&qtto=2002-12-31#result"

In [48]:
source, number_search_result=crawl_search_source(search_url)
soup = BeautifulSoup(source,'lxml')
set_url=find_links_from_source(soup, number_search_result)
save_pickle("links", set_url)
crawl_search_result(set_url)

5 0 3
total number of links: 2560
02ROME1 1 out of 2560 jobs done
02HARARE25 2 out of 2560 jobs done
02ROME17 3 out of 2560 jobs done
02ROME19 4 out of 2560 jobs done
02VATICAN24 5 out of 2560 jobs done
02VATICAN25 6 out of 2560 jobs done
02ABUJA14 7 out of 2560 jobs done
02ROME42 8 out of 2560 jobs done
02HARARE38 9 out of 2560 jobs done
02VATICAN57 10 out of 2560 jobs done
02ROME66 11 out of 2560 jobs done
02ABUJA32 12 out of 2560 jobs done
02HARARE44 13 out of 2560 jobs done
02ROME67 14 out of 2560 jobs done
02HARARE50 15 out of 2560 jobs done
02ROME73 16 out of 2560 jobs done
02VATICAN76 17 out of 2560 jobs done
02ROME81 18 out of 2560 jobs done
02VATICAN83 19 out of 2560 jobs done
02ABUJA38 20 out of 2560 jobs done
02HARARE61 21 out of 2560 jobs done
02HARARE63 22 out of 2560 jobs done
02JAKARTA97 23 out of 2560 jobs done
02HARARE81 24 out of 2560 jobs done
02VATICAN126 25 out of 2560 jobs done
02ROME134 26 out of 2560 jobs done
02HARARE90 27 out of 2560 jobs done
02HARARE95 28 ou

KeyboardInterrupt: 